# Food prediction model: L1

In [ ]:
# import packages 
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers


from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils.vis_utils import plot_model


import cv2
import os
import gc
import random
import matplotlib.image as mpimg

## Provide a model name

In [ ]:
model_name = 'J1'

## Load labels and paths

In [ ]:
# file location
train_label_path = '/kaggle/input/food-recognition-challenge-2021/train_labels.csv'
train_image_path = '/kaggle/input/food-recognition-challenge-2021/train_set/train_set/'
test_image_path= '/kaggle/input/food-recognition-challenge-2021/test_set'
test_results_sample = '/kaggle/input/food-recognition-challenge-2021/sample.csv'

# importing files
df_train_labels = pd.read_csv(train_label_path, sep=',')
df_test_labels = pd.read_csv(test_results_sample, sep=',')

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
#df_test_labels.head()

## Image sample

## Data pre-procession

In [ ]:
# Adding the file paths to the dataframe for train
path_list = []
for i in range(len(df_train_labels)):
    path = '{}'.format(df_train_labels['img_name'][i])
    path_list.append(path)

    
df_train_labels['path'] = path_list
df_train_labels['label'] = df_train_labels['label'].astype(str)

del path_list

# Adding the file paths to the dataframe for test
path_list = []
for i in range(len(df_test_labels)):
    path = '{}'.format(df_test_labels['img_name'][i])
    path_list.append(path)

    
df_test_labels['path'] = path_list
df_test_labels['label'] = df_test_labels['label'].astype(str)


## Train and validation set

In [ ]:
image_train, image_validation = train_test_split(df_train_labels,
                                                 test_size=0.1, 
                                                 random_state=1, 
                                                 stratify=df_train_labels['label'])

# Loading images training and validation

In [ ]:
# setting image parameters
img_height = 256
img_width = 256
image_size = (img_height,img_width)
batch_size = 35

# defining Generators
datagen = ImageDataGenerator(rotation_range=12, vertical_flip = True, horizontal_flip = True, zoom_range = 0.2, shear_range=10)
valigen=ImageDataGenerator()
test = ImageDataGenerator()

# parameters train data
train_images = datagen.flow_from_dataframe(
    image_train,
    directory = train_image_path,
    class_mode='categorical',
    x_col='path',
    y_col='label',
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size
)


#  parameters validation data
validation_images = valigen.flow_from_dataframe(
    image_validation,
    directory = train_image_path,
    class_mode='categorical',
    x_col='path',
    y_col='label',
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size
)


#  parameters test data
test_generator = test.flow_from_directory(
        test_image_path,
        target_size=image_size,
        batch_size=batch_size,
        shuffle=False
)

## Model design

In [ ]:
# Number of output layers/classes
num_classes = 80

#Setting the input size
input_shape = (img_height,img_width,3)

model = tf.keras.applications.EfficientNetB4(
    include_top=False,
     weights="imagenet",
#     input_tensor=None,
    input_shape=input_shape,
    pooling="avg",
    classes=num_classes
)

# Since we load pre-trained model we must specify first layer as non-trainable
# model.layers[0].trainable = True
model.trainable = True

inputs = model.input

x = tf.keras.layers.Dense(128, activation='relu')(model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(80, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)



# compiling model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
# model overview
model.summary()

## Model training

In [ ]:
from sklearn.utils import class_weight 

class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_images.classes), 
            train_images.classes)

train_class_weights = dict(enumerate(class_weights))
print('')

In [ ]:
# setting the location for train results
checkpoint_path = 'training_1/cp.ckpt'
#checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# setting the number of training iterations
epochs= 45

# # saving substeps
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  monitor = 'val_accuracy',
#                                                  save_best_only=True,
#                                                  verbose=1)

chkp = tf.keras.callbacks.ModelCheckpoint('EffNetB4_GiveUsMore.h5', monitor='val_accuracy', save_best_only=True)

# initiate training
history = model.fit(
    train_images,
    validation_data = validation_images,
    epochs=epochs,
    class_weight=train_class_weights,
    callbacks=[chkp]
)

model.save("J1.h5")

## Accuracy plots

In [ ]:
# subsetting all hyper results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

# plotting Training and Validation Accuracy
plt.figure(figsize=(14, 14))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# plotting the Training and Validation Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
plt.savefig('testim.png')

## Predicting on the testset

In [ ]:
def predict_for_test(model):
    '''
    This functions tests the model on the test set and 
    automaticaly maps the resulting labels to a .csv file.
    
    '''
    # getting labels
    labels = (train_images.class_indices)
    labels = dict((v,k) for k,v in labels.items())

    # predicting on the test set
    preds = model.predict(test_generator)
    preds_cls_idx = preds.argmax(axis=-1)
    predictions = [labels[k] for k in preds_cls_idx]

    #m apping predictions and save as df
    filenames=test_generator.filenames
    filenames = [x.replace('test_set/', '') for x in filenames]
    results=pd.DataFrame({"img_name":filenames,
                          "label":predictions})
    return results 

test_results_pr = predict_for_test(model)
test_results = df_test_labels.merge(test_results_pr, how='left', on='img_name')
test_results = test_results[['path', 'label_y']].rename(columns = {'label_y':'label', 'path':'img_name'})


# writing pridictions to csv
test_results.to_csv("sample_submission_EFFB4.csv", index=False)


In [ ]:
# showing the prediction dist
plt.figure(figsize=(20,15))
sns.countplot(x='label', data=test_results).set(title='Count per label')



plt.show()


# plt.savefig('model_1_label_dist.png')

In [ ]:
# printing the predictions
test_results